## 核心概念

多个 Tensor 变量可能指向内存中同一个 Storage。修改其中一个 Tensor 的数值，另一个也会随之改变，因为它们共享物理内存。

## 导入必要的库

In [1]:
import torch

## 创建基础 Tensor 并查看 Storage

In [2]:
# 1. 创建一个基础 Tensor
t_orig = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.int32)
t_orig = torch.tensor([[1, 258, 3], [4, 5, 6]], dtype=torch.int32)

# 2. 获取其底层的 Storage（在新版 PyTorch 中通常访问 untyped_storage）
storage = t_orig.untyped_storage()
print(type(storage))

print(f"原始 Tensor:\n{t_orig}")
print(f"Storage 内容 (字节流): {list(storage)}")

<class 'torch.storage.UntypedStorage'>
原始 Tensor:
tensor([[  1, 258,   3],
        [  4,   5,   6]], dtype=torch.int32)
Storage 内容 (字节流): [1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 4, 0, 0, 0, 5, 0, 0, 0, 6, 0, 0, 0]


## 理解 Storage 的字节表示

这里看到的 [1, 0, 0, 0] 实际上就是一个完整的整数 1。

### 1. 为什么会有那么多 0？（数据宽度）

我们在创建 Tensor 时指定了 `dtype=torch.int32`。Int32 意味着每个整数占用 32 位（bits）。
- 32 位 = 4 字节（Bytes）。
- 即使你存储的数字只是 1，它在内存中也必须霸占 4 个房间（字节）。

对于数字 1 来说，它的二进制在数学上是：
```
00000000 00000000 00000000 00000001
```

这里面有 3 个字节全是 0，只有最后 1 个字节有值。这就是你看到那一串 0 的原因。

### 2. 为什么 1 在最前面？（小端序 Little Endian）

你可能觉得数字 1 的存储应该是：[0, 0, 0, 1] (这是符合人类阅读习惯的顺序，叫大端序 Big Endian)

但在现代绝大多数 CPU（包括你的 Intel/AMD 芯片或 Apple M 系列芯片）中，内存采用的是 **小端序（Little Endian）**。

规则：低位字节（原本在最右边的）存在低地址（内存的最开头）。

所以，当你打印 Storage（它只是单纯地把内存里的字节一个个读出来打印成十进制）时，你看到的就是：1, 0, 0, 0

In [3]:
print(
    f"Storage 元素视点 (Float/Int): {storage.tolist()}"
)  # 注意：这显示原始字节转换后的列表

Storage 元素视点 (Float/Int): [1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 4, 0, 0, 0, 5, 0, 0, 0, 6, 0, 0, 0]


## 创建视图并验证内存共享

In [4]:
# 3. 创建一个视图 (View)
# view 操作不复制数据，只创建新元数据
t_view = t_orig.view(6)
print(f"\n视图 Tensor:\n{t_view}")


视图 Tensor:
tensor([  1, 258,   3,   4,   5,   6], dtype=torch.int32)


In [5]:
# 4. 验证物理内存共享
print(f"\n原始 Tensor 内存地址: {t_orig.data_ptr()}")
print(f"视图 Tensor 内存地址: {t_view.data_ptr()}")
print(f"两者是否指向同一内存地址: {t_orig.data_ptr() == t_view.data_ptr()}")


原始 Tensor 内存地址: 36438383808
视图 Tensor 内存地址: 36438383808
两者是否指向同一内存地址: True


## 修改视图，观察原始 Tensor 的变化

In [6]:
print("\n>>> 修改视图 t_view[0] = 999")
t_view[0] = 999
print(f"原始 Tensor (t_orig) 也变了:\n{t_orig}")


>>> 修改视图 t_view[0] = 999
原始 Tensor (t_orig) 也变了:
tensor([[999, 258,   3],
        [  4,   5,   6]], dtype=torch.int32)
